In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import IPython.display as ipd

wav_path = "lotek_owl_data/test.wav"
ipd.Audio(wav_path)

In [ ]:
from data_utils import display_audio_file


display_audio_file(wav_path)

In [ ]:
from data_utils import display_melspec, display_zero_crossings


display_zero_crossings()

In [ ]:
from data import load_data


batch_sz = 512
debug = False
spec_height = 750
owlet_train, owlet_test, owlet_dataset = load_data(
    "lotek_owl_data",
    train_test_split=[1.0, 0],
    batch_sz=batch_sz,
    debug=debug,
    spec_height=spec_height
)

---
# How does it work?

<img src="./img/explain_model.png" alt="diagram" width="1200">

---

<img src="./img/explain_loss.png" alt="loss" width="600">

---

# This is where the training code would go, but in the interest of time ...
---

In [ ]:
import torch


def create_embeds(model, dataloader):
    embeds = []
    specs = []
    specs_og = []
    for batch in dataloader:
        data_specs, og_specs = batch
        specs_og += og_specs.unbind()
        specs += data_specs.unbind()
        data_specs = data_specs.cuda()
        embeds_batch = model(data_specs.cuda())
        embeds.append(embeds_batch.detach().cpu())
    embeds = torch.cat(embeds)
    return embeds, specs, specs_og

In [ ]:
from model import OwlNet


drop = 0.0
embed_sz = 64
spec_height = 750

model_name = "model_3584.datapoints_105.epochs.pth"
owlnet_dict = torch.load(model_name)
owlnet = OwlNet(embed_sz, drop).cuda()
owlnet.load_state_dict(owlnet_dict)
owlnet.eval()
pass


In [ ]:
from utils import VisualiserInteractive
from data import CollateFunc


collate_func = CollateFunc(spec_height=spec_height)
embeddings, melspecs, melspecs_og = create_embeds(owlnet, owlet_train)
vis = VisualiserInteractive(embeddings, melspecs, melspecs_og)
vis.show()

In [ ]:
import time
import torch.nn.functional as F
from data import get_verification_dataloader


auto_run = True

def toggle_run():
    global auto_run
    auto_run = not auto_run


total_ds_size = 3375
hop_size = 20
for start in range(0, total_ds_size, hop_size):
    hop_size = min(total_ds_size - start, hop_size)
    indices = [start, start + hop_size]
    verification_dl = get_verification_dataloader(owlet_dataset, indices, collate_func)
    validation_embeds, _, _ = create_embeds(owlnet, verification_dl)
    vis.pop_verification_trace()
    validation_embeds = F.normalize(validation_embeds, p=2, dim=1)
    vis.add_points(validation_embeds, 'x', 20)

    # while not auto_run:
    #     key = input("Press Enter to step, or type 'run' to continue automatically: ").strip()
    #     if key.lower() == "run":
    #         toggle_run() 

